# FOR TEMP
## Obiettivi Prefissati
- Monitoraggio in real time dalla temperatura a bassa quota
- Previsione a breve termine della temperatura a bassa quota
- Acquisizione dei dati necessari al funzionamento senza obbligo alla sottoscrizione di servizi a pagamento

## Cos'è ForTemp

ForTemp è un progetto che sfrutta tecnologie distribuite per il monitoraggio dei dati meteorologici e per la previsione a breve termine (1h) della temperatura a bassa quota (2m).  
I dati provengono da dataset eterogenei che forniscono metriche diverse:  
- Da ERA5 Single Level si ottengono, ad esempio, medie satellitari su una griglia con risoluzione di circa 32 km.  
- Tramite l'API Current Weather di OpenWeather è possibile accedere a dati puntuali provenienti da varie stazioni.

L'interpolazione dei dati permette di creare un modello che, nella modalità batch, sfrutta i dati storici di ERA5 Single Level, mentre per l'analisi in tempo reale impiega i valori offerti da OpenWeather per prevedere la temperatura a bassa quota.  
Il modello, pur essendo stato addestrato su dati medi storici e testato con misurazioni in tempo reale, riesce a generare previsioni puntuali. Tuttavia, per ottimizzarne le stime e verificarne l'efficacia nell'approccio ibrido, sono necessarie ulteriori analisi e sperimentazioni.

ForTemp offre due modalità d'uso:  
1. Utilizzo immediato tramite un modello preaddestrato basato sui dati storici di ERA5 Single Level.  
2. Modalità (beta) per addestrare un modello personalizzato in base al proprio bound di coordinate, utilizzando sempre i dati di ERA5 Single Level.  

Indipendentemente dalla modalità scelta, il modello richiede una fase di warm-up (circa 4 ore) prima di iniziare a generare previsioni in tempo reale.


##  API richieste per il funzionamento

### OpenWeather  
OpenWeather è un'azienda che fornisce dati meteorologici attraverso API basate su cloud. Offre previsioni del tempo, dati storici e analisi climatiche utilizzando modelli meteorologici avanzati e dati raccolti da satelliti, radar e stazioni meteorologiche. Le API fornite gratuitamente (con limite di richieste mensili) previa registrazione:
- Current weather API
- GeoCoding API

### OpenElevationAPI 
Open-Elevation è un servizio open-source che fornisce dati sull'altitudine di un punto geografico basandosi su coordinate (latitudine e longitudine).Open-Elevation si basa su dati pubblici e può essere usato gratuitamente con un limite di richieste per il piano base senza registrazione.

### CDS 
Il ECMWF (European Centre for Medium-Range Weather Forecasts) è un'organizzazione intergovernativa che fornisce previsioni del tempo globali e dati climatici avanzati, con particolare attenzione alle previsioni a medio e lungo termine.
Il CDS (Climate Data Store) è la piattaforma online gestita da ECMWF del Copernicus Climate Change Service (C3S)  che permette di accedere gratuitamente a una vasta gamma di dati climatici e meteorologici, inclusi quelli di ERA5, sotto licenza ECMWF.
I dataset utilizzati nel modello al fine del training sono:
- ERA5/ERA5T Single Level


## Tecnologie utilizzate

- Script Python
- Docker , docker compose
- Logstash
- Kafka 
- Kafka Streaming
- Spark Structured Stream
- Spark
- ElasticSearch
- Kibana
- Crond

![Alt text ](image/pipeline.png)

## Come funziona 



### [Opzionale] Addestramento modello
- Download grib dataset da era5 single level e pressure level:
    - scrivendosi i json delle richieste ed in seguito utilizzando script Python per il download del dataset da ecwma cds
    - oppure tramite web interface all'indirizzo: https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=download
- script Python per la conversione del dataset da ecwma cds (.grib) in .csv
- Spark per il training della regressione lineare



### Modello pre-addestrato
- Dati utilizzati:
    Dati proventienti da ERA5/ERA5T per la zona: "37.7, 14.5, 37.3, 15.3" per l'arco temporale che procede dal 1-1-2020 al 31-12-2024

- Modello utilizzato:
    Machine learning in Regressione lineare

### Pipeline di Monitoraggio con modello pre-addestrato per il forecast
- tool python che consente di:
    - aggiungere città da monitorare
    - aggiungere costante altitudine alle città ottenuta da OpenElevation
    - ottenere variabili metereologiche ottenute da OpenWeather, convertirle nel formato appropriato alla pipeline ed mandarle ad uno specifico server 
- Logstash: ingestion manager per kafka ed Elastick
- Kafka: Sistema di messaggistica distribuibile e scalabile in realtime
- Kafka Stream: Utillizzato per aggregare i dati ricevuti. In questa fase di worm up avviene un'aggregazione dei dati in finestre di 4 ore
- Spark Structured Stream: Utilizzato per l'elaborazione dei messaggi aggregati in un modello di regressione lineare per ottenere il target temperaturatura futura
- ElastickSearch: Indicizzazione dei dati
- Kibana: Presentazione indici


## Utilizzi reali

Nel campo ove la temperatura è cruciale con la regolazione manuale di riscaldamenti e refrigeratori per la diminuizione degli impatti economici ed ambientali ; come in ospedali, serre ma anche ove non è cruciale come case e infrastrutture fisse ad alto spreco di rendimento

## Possibili miglioramenti futuri

- inseririre nuovi processi per il download  da ecwma e training di tali dati per ottenere sempre un modello che utilizza dati storici più recenti 
- integrazione con input in real time da cabina metereologica homesafe al posto di openweather.
- integrazione in output con un sistema di notificazione per l'accensione o spegnimento di impianti.
- distribuzione dei server con una orchestrazione e robustezza migliore utilizzando tecnoligie come K8s,Terraform ecc.

# Presentazione DEMO
![Alt text](image/docker_meme.webp)
## Architettura:

- ### Google Cloud Plattaform
    - Compute Engine #1
        - e2-standard-2 (2 vCPUs, 8 GB Memory)
        - 20gb Disco permanente con carico bilanciato
        - rete Google Premium, con ipforward disabilitato
        - firewall role non abilitato a condividere estreriormente
        - Comunicazione tramite tuneling ssh 
- ### Moodello
    - Pre-addestrato 
- ### Città Analizzate
    - Catania
    - Roma
    - Aosta
    - Palermo
    - Torino
    - Napoli
    - Milano
    - Trieste
    - Firenze
    - Genova
    - Bari
    - Cagliari
    - Venezia
    - Bolzano
    - Sanremo
    - Bologna



## Credits

CDS

    The model of linear regression is generated using Copernicus Climate Change Service information 2020-2024 . Neither the European Commission nor ECMWF is responsible for any use that may be made of the Copernicus information or data it contains.

    Citing the data:

    Hersbach, H., Bell, B., Berrisford, P., Biavati, G., Horányi, A., Muñoz Sabater, J., Nicolas, J., Peubey, C., Radu, R., Rozum, I., Schepers, D., Simmons, A., Soci, C., Dee, D., Thépaut, J-N. (2023): ERA5 hourly data on single levels from 1940 to present. Copernicus Climate Change Service (C3S) Climate Data Store (CDS), DOI: 10.24381/cds.adbb2d47 (Accessed on 17-2-2025)

    Citing the web catalogue entry:

    Copernicus Climate Change Service, Climate Data Store, (2023): ERA5 hourly data on single levels from 1940 to present. Copernicus Climate Change Service (C3S) Climate Data Store (CDS), DOI: 10.24381/cds.adbb2d47 (Accessed on 17-2-2025)

### Docs

- OpenWeather API Current Weather https://openweathermap.org/current
- Openweather API GeoCoding https://openweathermap.org/current#geocoding
- OpenElevation API Docs https://open-elevation.com/
- CDS API https://cds.climate.copernicus.eu/how-to-api
- EcCodes https://confluence.ecmwf.int/display/ECC/documentation
- Pandas :https://pandas.pydata.org/docs/
- Docker https://docs.docker.com/
- Docker compose https://docs.docker.com/compose/
- Kafka https://kafka.apache.org/documentation/
- Kafka Stream https://kafka.apache.org/documentation/streams/
- Spark https://archive.apache.org/dist/spark/docs/3.4.4/
- Spark sstructured streaming https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html
- PySpark https://spark.apache.org/docs/latest/api/python/index.html
- Crond https://www.linux.org/docs/man8/cron.html
- Logstash https://www.elastic.co/guide/en/logstash/current/index.html
- Elasticksearch https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html
- Kibana https://www.elastic.co/guide/en/kibana/current/index.html


